In [ ]:
# Python
import os
import sys
import time
import random
import itertools
from tqdm import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict

# NumPy and PyTorch
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [ ]:
class PropagationPath:
    def __init__(self):
        self.points = []
        self.interaction_types = []   # 0 - initial transmitter (radio antenna)   1 - final point (user)   2 - specular reflection   3 - diffraction around the edge
        self.path_gain_db = 0   # dB = 10 * log10(I/I0)   -20 dB = 100 times weaker   -90 dB = 1000000000 times weaker
        self.hash = 0   # hashed set of interactions (objects / surfaces)

offset = 0
def read_int():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.int32)[0]
def read_uint():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.uint32)[0]
def read_float():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.float32)[0]

def read_file(filename: str):
    global offset, bytes
    offset = 0
    bytes = np.fromfile(filename, dtype=np.uint8)

    transmitter_count = read_int()
    paths = []
    for tx in range(transmitter_count):
        receiver_count = read_int()
        paths.append([])
        for rx in range(receiver_count):
            path_count = read_int()
            paths[tx].append([])
            for p in range(path_count):
                path = PropagationPath()
                point_count = read_int()
                for _ in range(point_count):
                    x = read_float()
                    y = read_float()
                    z = read_float()
                    path.points.append((x, y, z))
                for _ in range(point_count):
                    interaction = read_int()
                    path.interaction_types.append(interaction)
                path.path_gain_db = read_float()
                path.hash = read_uint()
                paths[tx][rx].append(path)
    return paths

In [ ]:
class ConvChain(nn.Module):
    """A simple stack of convolution layers.

    Args:
        ninputs(int): number of input channels.
        noutputs(int): number of output channels.
        ksize(int): size of all the convolution kernels.
        width(int): number of channels per intermediate layer.
        depth(int): number of intermadiate layers.
        stride(int): stride of the convolution.
        pad(bool): if True, maintains spatial resolution by 0-padding,
            otherwise keep only the valid part.
        normalize(bool): applies normalization if True.
        normalization_type(str): either batch or instance.
        output_type(str): one of linear, relu, leaky_relu, tanh, elu.
        activation(str): one of relu, leaky_relu, tanh, elu.
        weight_norm(bool): applies weight normalization if True.
    """
    def __init__(self, ninputs, noutputs, ksize=3, width=64, depth=3, stride=1,
                 pad=True, normalize=False, normalization_type="batch",
                 output_type="linear", activation="relu", weight_norm=True):
        super(ConvChain, self).__init__()

        if depth <= 0:
            LOG.error("ConvChain should have non-negative depth.")
            raise ValueError("negative network depth.")

        if pad:
            padding = ksize//2
        else:
            padding = 0

        layers = []
        for d in range(depth-1):
            if d == 0:
                _in = ninputs
            else:
                _in = width
            layers.append(
                ConvChain._ConvBNRelu(_in, ksize, width, normalize=normalize,
                                      normalization_type=normalization_type,
                                      padding=padding, stride=stride,
                                      activation=activation,
                                      weight_norm=weight_norm))

        # Last layer
        if depth > 1:
            _in = width
        else:
            _in = ninputs

        conv = nn.Conv2d(_in, noutputs, ksize, bias=True, padding=padding)
        if weight_norm:
            conv = nn.utils.weight_norm(conv)
        conv.bias.data.zero_()
        if output_type == "elu" or output_type == "softplus":
            nn.init.xavier_uniform_(
                conv.weight.data, nn.init.calculate_gain("relu"))
        else:
            nn.init.xavier_uniform_(
                conv.weight.data, nn.init.calculate_gain(output_type))
        layers.append(conv)

        # Rename layers
        for im, m in enumerate(layers):
            if im == len(layers)-1:
                name = "prediction"
            else:
                name = "layer_{}".format(im)
            self.add_module(name, m)

        if output_type == "linear":
            pass
        elif output_type == "relu":
            self.add_module("output_activation", nn.ReLU(inplace=True))
        elif output_type == "leaky_relu":
            self.add_module("output_activation", nn.LeakyReLU(inplace=True))
        elif output_type == "sigmoid":
            self.add_module("output_activation", nn.Sigmoid())
        elif output_type == "tanh":
            self.add_module("output_activation", nn.Tanh())
        elif output_type == "elu":
            self.add_module("output_activation", nn.ELU())
        elif output_type == "softplus":
            self.add_module("output_activation", nn.Softplus())
        else:
            raise ValueError("Unknon output type '{}'".format(output_type))

    def forward(self, x):
        for m in self.children():
            x = m(x)
        return x

    class _ConvBNRelu(nn.Module):
        """Helper class that implements a simple Conv-(Norm)-Activation group.

        Args:
            ninputs(int): number of input channels.
            ksize(int): size of all the convolution kernels.
            noutputs(int): number of output channels.
            stride(int): stride of the convolution.
            pading(int): amount of 0-padding.
            normalize(bool): applies normalization if True.
            normalization_type(str): either batch or instance.
            activation(str): one of relu, leaky_relu, tanh, elu.
            weight_norm(bool): if True applies weight normalization.
        """
        def __init__(self, ninputs, ksize, noutputs, normalize=False,
                     normalization_type="batch", stride=1, padding=0,
                     activation="relu", weight_norm=True):
            super(ConvChain._ConvBNRelu, self).__init__()

            if activation == "relu":
                act_fn = nn.ReLU
            elif activation == "leaky_relu":
                act_fn = nn.LeakyReLU
            elif activation == "tanh":
                act_fn = nn.Tanh
            elif activation == "elu":
                act_fn = nn.ELU
            else:
                LOG.error("Incorrect activation %s", activation)
                raise ValueError("activation should be one of: "
                                 "relu, leaky_relu, tanh, elu")

            if normalize:
                print("nrm", normalization_type)
                conv = nn.Conv2d(ninputs, noutputs, ksize,
                                 stride=stride, padding=padding, bias=False)
                if normalization_type == "batch":
                    nrm = nn.BatchNorm2d(noutputs)
                elif normalization_type == "instance":
                    nrm = nn.InstanceNorm2D(noutputs)
                else:
                    LOG.error("Incorrect normalization %s", normalization_type)
                    raise ValueError(
                        "Unkown normalization type {}".format(
                            normalization_type))
                nrm.bias.data.zero_()
                nrm.weight.data.fill_(1.0)
                self.layer = nn.Sequential(conv, nrm, act_fn())
            else:
                conv = nn.Conv2d(ninputs, noutputs, ksize,
                                 stride=stride, padding=padding)
                if weight_norm:
                    conv = nn.utils.weight_norm(conv)
                conv.bias.data.zero_()
                self.layer = nn.Sequential(conv, act_fn())

            if activation == "elu":
                nn.init.xavier_uniform_(
                    conv.weight.data, nn.init.calculate_gain("relu"))
            else:
                nn.init.xavier_uniform_(
                    conv.weight.data, nn.init.calculate_gain(activation))

        def forward(self, x):
            out = self.layer(x)
            return out


class Autoencoder(nn.Module):
    """A U-net style autoencoder.

    Args:
        ninputs(int): number of input channels.
        noutputs(int): number of output channels.
        ksize(int): size of all the convolution kernels.
        width(int): number of channels per intermediate layer at the finest
            scale.
        num_levels(int): number of spatial scales.
        num_convs(int): number of conv layers per scale.
        max_width(int): max number of features per conv layer.
        increase_factor(float): each coarsest scale increases the number of
            feature channels by this factor, up to `max_width`.
        normalize(bool): applies normalization if True.
        normalization_type(str): either batch or instance.
        output_type(str): one of linear, relu, leaky_relu, tanh, elu.
        activation(str): one of relu, leaky_relu, tanh, elu.
    """
    def __init__(self, ninputs, noutputs, ksize=3, width=64, num_levels=3,
                 num_convs=2, max_width=512, increase_factor=1.0,
                 normalize=False, normalization_type="batch",
                 output_type="linear",
                 activation="relu", pooling="max"):
        super(Autoencoder, self).__init__()

        next_level = None
        for lvl in range(num_levels-1, -1, -1):
            n_in = min(int(width*(increase_factor)**(lvl-1)), max_width)
            w = min(int(width*(increase_factor)**(lvl)), max_width)
            n_us = min(int(width*(increase_factor)**(lvl+1)), max_width)
            n_out = w
            o_type = activation

            if lvl == 0:
                n_in = ninputs
                o_type = output_type
                n_out = noutputs
            elif lvl == num_levels-1:
                n_us = None

            next_level = Autoencoder._Level(
                n_in, n_out, next_level=next_level, num_us=n_us,
                ksize=ksize, width=w, num_convs=num_convs,
                output_type=o_type, normalize=normalize,
                normalization_type=normalization_type,
                activation=activation, pooling=pooling)

        self.add_module("net", next_level)

    def forward(self, x):
        return self.net(x)

    class _Level(nn.Module):
        """One scale of the autoencoder processor.

        Args:
            num_inputs(int): number of input channels.
            num_outputs(int): number of output channels.
            next_level(Autoencoder._Level or None): the coarser level after
                this one, or None if this is the coarsest level.
            num_us(int): number of features in the upsampling branch.
            ksize(int): size of all the convolution kernels.
            width(int): number of channels per intermediate layer at the finest
                scale.
            num_convs(int): number of conv layers per scale.
            output_type(str): one of linear, relu, leaky_relu, tanh, elu.
            normalize(bool): applies normalization if True.
            normalization_type(str): either batch or instance.
            pooling(str): type of downsampling operator: "max", "average" or
                "conv".
            activation(str): one of relu, leaky_relu, tanh, elu.
        """
        def __init__(self, num_inputs, num_outputs, next_level=None,
                     num_us=None, ksize=3, width=64, num_convs=2,
                     output_type="linear", normalize=True,
                     normalization_type="batch", pooling="max",
                     activation="relu"):
            super(Autoencoder._Level, self).__init__()

            self.is_last = (next_level is None)

            if self.is_last:
                self.left = ConvChain(
                    num_inputs, num_outputs, ksize=ksize, width=width,
                    depth=num_convs, stride=1, pad=True,
                    normalize=normalize, normalization_type=normalization_type,
                    output_type=output_type)
            else:
                assert num_us is not None

                self.left = ConvChain(
                    num_inputs, width, ksize=ksize, width=width,
                    depth=num_convs, stride=1, pad=True, normalize=normalize,
                    normalization_type=normalization_type,
                    output_type=activation, activation=activation)
                if pooling == "max":
                    self.downsample = nn.MaxPool2d(2, 2)
                elif pooling == "average":
                    self.downsample = nn.AvgPool2d(2, 2)
                elif pooling == "conv":
                    self.downsample = nn.Conv2d(width, width, 2, stride=2)
                else:
                    raise ValueError("unknown pooling'{}'".format(pooling))

                self.next_level = next_level
                self.right = ConvChain(
                    num_us + width, num_outputs, ksize=ksize, width=width,
                    depth=num_convs, stride=1, pad=True, normalize=normalize,
                    normalization_type=normalization_type,
                    output_type=output_type)

        def forward(self, x):
            left = self.left(x)
            if self.is_last:
                return left

            ds = self.downsample(left)
            next_level = self.next_level(ds)
            us = F.interpolate(
                next_level, size=left.shape[-2:], mode='bilinear',
                align_corners=False)
            # Concat skip connection
            concat = th.cat([us, left], 1)
            output = self.right(concat)
            return output

In [120]:
def init_data(paths, batch_size, ratio):

    all_data = []

    for i in range(len(paths)):
      for j in range(len(paths[0])):
        if len(paths[i][j]) != 0:
          for k in paths[i][j]:
            arr = []
            c = 0
            for l in range(len(k.points)):
              c += 1
              for m in range(len(k.points[l])):
                arr.append(double(k.points[l][m]))
            for l in range((5-c)*3, 0, -1):
              arr.append(0)
            for l in range(len(k.interaction_types)):
              arr.append(double(k.interaction_types[l]))
            for l in range(5-c, 0, -1):
              arr.append(0)
            arr.append(k.path_gain_db)
            all_data.append(arr)

    numpy_data = np.array(all_data)

    #rnd_int = np.random.randint(0, len(all_paths))

    datasets = {}
    datasets['train'] = np.array(numpy_data[:int(len(numpy_data) * ratio)])
    datasets['val'] = np.array(numpy_data[int(len(numpy_data) * ratio):])

    # Shuffle data
    dataset_length = len(datasets['train'])
    indices = list(range(dataset_length))
    np.random.shuffle(indices)
    sampler = torch.utils.data.SubsetRandomSampler(indices)

    # Create DataLoaders using these samplers
    dataloaders = {
        'train': torch.utils.data.DataLoader(
            datasets['train'],
            batch_size=batch_size,
            sampler=sampler,
            num_workers=2,
            pin_memory=False,
        ),
        'val': torch.utils.data.DataLoader(
            datasets['val'],
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=False,
        )
    }

    return datasets, dataloaders

In [45]:
class Interface():
    def __init__(self, model):
        self.model = model

    def to_train_mode(self):
        for model_name in self.models:
            self.models[model_name].train()
            assert 'optim_' + model_name in self.optims, '`optim_%s`: an optimization algorithm is not defined.'%(model_name)

    def train_batch(self, batch, grad_hook_mode=False):
        out_manif = None
        self.model.zero_grad()
        p_buffer = self.model(batch)
        """ Feature disentanglement """
        """
            _, _, c, _, _ = p_buffers['diffuse'].shape
            assert c >= 2
            if self.disentanglement_option == 'm11r11':
                out_manif = p_buffers
            elif self.disentanglement_option == 'm10r01':
                out_manif = {
                        'diffuse': p_buffers['diffuse'][:,:,c//2:,...],
                        'specular': p_buffers['specular'][:,:,c//2:,...]
                }
                p_buffers = {
                        'diffuse': p_buffers['diffuse'][:,:,:c//2,...],
                        'specular': p_buffers['specular'][:,:,:c//2,...]
                }
            elif self.disentanglement_option == 'm11r01':
                out_manif = p_buffers
                p_buffers = {
                        'diffuse': p_buffers['diffuse'][:,:,:c//2,...],
                        'specular': p_buffers['specular'][:,:,:c//2,...]
                }
            elif self.disentanglement_option == 'm10r11':
                out_manif = {
                        'diffuse': p_buffers['diffuse'][:,:,c//2:,...],
                        'specular': p_buffers['specular'][:,:,c//2:,...]
                }

            p_var_diffuse = p_buffers['diffuse'].var(1).mean(1, keepdims=True).detach()
            p_var_diffuse /= p_buffers['diffuse'].shape[1] # spp
            p_var_specular = p_buffers['specular'].var(1).mean(1, keepdims=True).detach()
            p_var_specular /= p_buffers['specular'].shape[1]

            # make a new batch
            batch = {
                'target_total': batch['target_total'],
                'target_diffuse': batch['target_diffuse'],
                'target_specular': batch['target_specular'],
                'kpcn_diffuse_in': torch.cat([batch['kpcn_diffuse_in'], p_buffers['diffuse'].mean(1), p_var_diffuse], 1),
                'kpcn_specular_in': torch.cat([batch['kpcn_specular_in'], p_buffers['specular'].mean(1), p_var_specular], 1),
                'kpcn_diffuse_buffer': batch['kpcn_diffuse_buffer'],
                'kpcn_specular_buffer': batch['kpcn_specular_buffer'],
                'kpcn_albedo': batch['kpcn_albedo'],
            }

        self.models['dncnn'].zero_grad()
        out = self._regress_forward(batch)

        loss_dict = self._backward(batch, out, out_manif)

        if grad_hook_mode: # do not update this model
            return

        self._logging(loss_dict)

        self._optimization()
        """

    def _regress_forward(self, batch):
        return self.models['dncnn'](batch)

    def _backward(self, batch, out, p_buffers):
        assert 'radiance' in out
        assert 'diffuse' in out
        assert 'specular' in out

        total, diffuse, specular = out['radiance'], out['diffuse'], out['specular']
        loss_dict = {}
        tgt_total = crop_like(batch['target_total'], total)

        if self.train_branches: # training diffuse and specular branches
            tgt_diffuse = crop_like(batch['target_diffuse'], diffuse)
            L_diffuse = self.loss_funcs['l_diffuse'](diffuse, tgt_diffuse)

            tgt_specular = crop_like(batch['target_specular'], specular)
            L_specular = self.loss_funcs['l_specular'](specular, tgt_specular)

            loss_dict['l_diffuse'] = L_diffuse.detach()
            loss_dict['l_specular'] = L_specular.detach()

            if self.manif_learn:
                p_buffer_diffuse = crop_like(p_buffers['diffuse'], diffuse)
                L_manif_diffuse = self.loss_funcs['l_manif'](p_buffer_diffuse, tgt_diffuse)
                L_diffuse += L_manif_diffuse * self.w_manif

                p_buffer_specular = crop_like(p_buffers['specular'], specular)
                L_manif_specular = self.loss_funcs['l_manif'](p_buffer_specular, tgt_specular)
                L_specular += L_manif_specular * self.w_manif

                loss_dict['l_manif_diffuse'] = L_manif_diffuse.detach()
                loss_dict['l_manif_specular'] = L_manif_specular.detach()

            L_diffuse.backward()
            L_specular.backward()

            with torch.no_grad():
                L_total = self.loss_funcs['l_recon'](total, tgt_total)
                loss_dict['l_total'] = L_total.detach()
        else: # post-training the entire system
            L_total = self.loss_funcs['l_recon'](total, tgt_total)
            loss_dict['l_total'] = L_total.detach()
            L_total.backward()

        with torch.no_grad():
            loss_dict['rmse'] = self.loss_funcs['l_test'](total, tgt_total).detach()

        return loss_dict

    def get_epoch_summary(self, mode, norm):
        if mode == 'train':
            print('[][][]', end=' ')
            for key in self.m_losses:
                if key == 'm_val':
                    continue
                tr_l_tmp = self.m_losses[key] / (norm * 2)
                tr_l_tmp *= 1000
                print('%s: %.3fE-3'%(key, tr_l_tmp), end='\t')
                self.m_losses[key] = torch.tensor(0.0, device=self.m_losses[key].device)
            print('')
            return -1.0
        else:
            return self.m_losses['m_val'].item() / (norm * 2)


In [131]:
class PathNet(nn.Module):
    """Path embedding network"""

    def __init__(self, ic, intermc=64, outc=3):
        super(PathNet, self).__init__()
        self.ic = ic
        self.intermc = intermc
        self.outc = outc
        self.final_ic = intermc + intermc

        self.embedding = ConvChain(ic, intermc, width=intermc, depth=3,
                ksize=1, pad=False)
        self.propagation = Autoencoder(intermc, intermc, num_levels=3,
                increase_factor=2.0, num_convs=3, width=intermc, ksize=3,
                output_type="leaky_relu", pooling="max")
        self.final = ConvChain(self.final_ic, outc, width=self.final_ic,
                depth=2, ksize=1, pad=False, output_type="relu")

    def __str__(self):
        return "PathNet i{}in{}o{}".format(self.ic, self.intermc, self.outc)

    def forward(self, samples):
        paths = samples
        bs, w = paths.shape

        flat = paths.contiguous().view([bs, w])
        flat = self.embedding(flat)
        print(1111)
        flat = flat.view([bs, spp, self.intermc, h, w])
        reduced = flat.mean(1)

        propagated = self.propagation(reduced)
        flat = torch.cat([flat.view([bs*spp, self.intermc, h, w]), propagated.unsqueeze(1).repeat(
            [1, spp, 1, 1, 1]).view(bs*spp, self.intermc, h, w)], 1)
        out = self.final(flat).view([bs, spp, self.outc, h, w])
        return out

In [121]:
path_file = 'eu_city_2x2_macro_306.bin'
paths = read_file(path_file)

# Set random seeds
np.random.seed(0)
batch_size = 10
train_val_ratio = 0.95

dataset, dataloaders = init_data(paths, batch_size, train_val_ratio)

In [132]:
n_in = 1
n_out = 1
model = PathNet(ic=n_in, outc=n_out)

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device('cpu')
print("Using {}.".format(device))

# Send the model to GPU
model = model.to(device)

interface = Interface(model)


Using cpu.


In [135]:
for inputs in dataloaders['train']:
  inputs = inputs.to(device)
  print(inputs)
  interface.train_batch(inputs, grad_hook_mode=False)

<class 'torch.Tensor'>


RuntimeError: ignored

In [ ]:
"""
params_to_update = model.parameters()
print("Params to learn:")
for name,param in model.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

optimizer = optim.Adam(params_to_update, lr=0.001)
"""